In [ ]:
import math

import numpy

import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
xrange = tf.linspace(-math.pi, math.pi, 201)
baseline_data = tf.math.sin(xrange)

def get_data():
    return tf.map_fn(lambda e: e + numpy.random.uniform(-0.1, 0.1), baseline_data)

for i in range(10):
    plt.plot(get_data(), color="green")

plt.show()